In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Display the dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
choice_2021 = pd.read_excel('data/2021.xlsx', sheet_name='Resultaten')
choice_2020 = pd.read_excel('data/2020.xlsx', sheet_name='Resultaten')
choice_2019 = pd.read_excel('data/2019.xlsx', sheet_name='Resultaten')
school_2021 = pd.read_excel('data/2021.xlsx', sheet_name='Klassen')
school_2020 = pd.read_excel('data/2020.xlsx', sheet_name='Klassen')
school_2019 = pd.read_excel('data/2019.xlsx', sheet_name='Klassen')

In [4]:
choice_2021 = choice_2021[choice_2021['Geplaatst op'] != 'Niet geplaatst']
choice_2020 = choice_2020[choice_2020['Geplaatst op'] != 'Niet geplaatst']
choice_2019 = choice_2019[choice_2019['Geplaatst op'] != 'Niet geplaatst']


In [5]:
choice_2021.drop(['Voorrang/Hardheid eerste voorkeur'], axis=1, inplace=True)
choice_2020.drop(['Voorrang/Hardheid eerste voorkeur'], axis=1, inplace=True)
choice_2019.drop(['Voorrang/Hardheid eerste voorkeur'], axis=1, inplace=True)


In [6]:
school_2021 = school_2021[school_2021['max_capacity'] > 0]

In [193]:
years = [2021, 2020, 2019]
choice_sets = [choice_2021, choice_2020, choice_2019]
school_sets = [school_2021, school_2020, school_2019]
for choices, year in zip(choice_sets, years):
    print('Choice data of {}: {}'.format(year, choices.shape))
for schools, year in zip(school_sets, years):
    print('School data of {}: {}'.format(year, schools.shape))

Choice data of 2021: (7976, 25)
Choice data of 2020: (7663, 23)
Choice data of 2019: (7568, 25)
School data of 2021: (142, 3)
School data of 2020: (132, 3)
School data of 2019: (130, 3)


In [194]:
for year, schools in zip(years, school_sets): 
    print(year, ': \n', schools.nunique())

2021 : 
 school           63
group           142
max_capacity     69
dtype: int64
2020 : 
 school           63
group           132
max_capacity     70
dtype: int64
2019 : 
 school           64
group           130
max_capacity     65
dtype: int64


In [195]:
choice_2021[~choice_2021['Voorkeur 1'].isin(school_2021.group)]

# empty --> every school preference matches the group name in school_2021 dataset

,Basisschool advies,Lotnummer,Geplaatst op,Positie,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19,Voorkeur 20,Voorkeur 21


In [196]:
choice_2021['Basisschool advies'].value_counts()

vwo            2290
havo/vwo       1239
havo           1206
vmbo-t         1126
vmbo-t/havo     796
vmbo-k          618
vmbo-b          389
vmbo-b/k        312
Name: Basisschool advies, dtype: int64

In [197]:
choice_2020['Basisschool advies'].value_counts().to_dict()

{'vwo': 2076,
 'havo': 1207,
 'vmbo-t': 1122,
 'havo/vwo': 1056,
 'vmbo-t/havo': 729,
 'vmbo-k': 512,
 'vmbo-b, met lwoo': 324,
 'vmbo-k, met lwoo': 191,
 'vmbo-b/k, met lwoo': 187,
 'vmbo-b/k': 149,
 'vmbo-b': 81,
 'vmbo-t, met lwoo': 29}

### Advice make-up over years

In [198]:
advice_2021 = choice_2021['Basisschool advies'].value_counts().to_dict()
advice_2020 = {
    'vwo': 2076,
    'havo': 1208,
    'vmbo-t': 1127+29,
    'havo/vwo': 1059,
    'vmbo-t/havo': 729,
    'vmbo-k': 512+191,
    'vmbo-b/k': 149+187,
    'vmbo-b': 81+324
}
advice_2019 = {
    'vwo': 2160,
    'havo': 1117,
    'vmbo-t': 1019+31,
    'havo/vwo': 1015,
    'vmbo-t/havo': 739,
    'vmbo-k': 496+214,
    'vmbo-b/k': 135+180,
    'vmbo-b': 92+382
}
advice_2018 = {
    'vwo': 2083,
    'havo': 1199,
    'vmbo-t': 990+46,
    'havo/vwo': 977,
    'vmbo-t/havo': 626,
    'vmbo-k': 402+303,
    'vmbo-b/k': 98+215,
    'vmbo-b': 55+426
}

In [199]:
advice_proportions = pd.DataFrame([advice_2021, advice_2020, advice_2019, advice_2018], index=[2021, 2020, 2019, 2018])
advice_proportions

,vwo,havo/vwo,havo,vmbo-t,vmbo-t/havo,vmbo-k,vmbo-b,vmbo-b/k
2021,2290,1239,1206,1126,796,618,389,312
2020,2076,1059,1208,1156,729,703,405,336
2019,2160,1015,1117,1050,739,710,474,315
2018,2083,977,1199,1036,626,705,481,313


In [200]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

edu_types = ['VWO', 'HAVO/VWO', 'HAVO', 'VMBO-T', 'VMBO-T/HAVO', 'VMBO-K', 'VMBO-B', 'VMBO-B/K']

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['2021', '2020', '2019', '2018'])
fig.add_trace(go.Pie(labels=edu_types, values=advice_proportions.iloc[0,:]), 1, 1)
fig.add_trace(go.Pie(labels=edu_types, values=advice_proportions.iloc[1,:]), 1, 2)
fig.add_trace(go.Pie(labels=edu_types, values=advice_proportions.iloc[2,:]), 1, 3)
# fig.add_trace(go.Pie(labels=edu_types, values=advice_proportions.iloc[3,:]), 1, 4)
fig.update_layout(title_text='Education types advised over years')

fig.show()
fig.write_image("images/advice.png")


### Positions landed

In [201]:
(choice_2021['Positie'].value_counts(normalize=True) * 100).round(2)

1     76.83
2     10.19
3      4.63
4      3.13
5      2.03
6      1.02
7      0.80
8      0.71
9      0.34
10     0.25
11     0.06
Name: Positie, dtype: float64

In [202]:
(choice_2020['Positie'].value_counts(normalize=True) * 100).round(2)

1     81.01
2      8.21
3      3.30
4      2.13
5      1.85
6      0.98
7      0.84
9      0.55
8      0.44
11     0.31
10     0.22
12     0.16
Name: Positie, dtype: float64

In [203]:
(choice_2019['Positie'].value_counts(normalize=True) * 100).round(2)


1     83.19
2      8.39
3      3.87
4      1.52
5      1.08
6      0.59
7      0.37
8      0.33
9      0.22
10     0.17
11     0.12
12     0.07
13     0.05
14     0.01
Name: Positie, dtype: float64

In [204]:
position_advice = choice_2021[['Positie', 'Basisschool advies']].value_counts().unstack()
position_advice.fillna(0, inplace=True)
position_advice['total'] = position_advice.sum(axis=1)
position_advice

Basisschool advies,havo,havo/vwo,vmbo-b,vmbo-b/k,vmbo-k,vmbo-t,vmbo-t/havo,vwo,total
Positie,,,,,,,,,
1,843.0,783.0,364.0,278.0,583.0,990.0,655.0,1632.0,6128.0
2,142.0,151.0,23.0,29.0,31.0,83.0,79.0,275.0,813.0
3,62.0,91.0,2.0,5.0,4.0,31.0,34.0,140.0,369.0
4,54.0,63.0,0.0,0.0,0.0,14.0,16.0,103.0,250.0
5,39.0,46.0,0.0,0.0,0.0,7.0,11.0,59.0,162.0
6,16.0,32.0,0.0,0.0,0.0,1.0,1.0,31.0,81.0
7,12.0,21.0,0.0,0.0,0.0,0.0,0.0,31.0,64.0
8,15.0,27.0,0.0,0.0,0.0,0.0,0.0,15.0,57.0
9,14.0,10.0,0.0,0.0,0.0,0.0,0.0,3.0,27.0


In [221]:
fig = px.bar(position_advice.iloc[:5,:-1], orientation='h', labels={'value': 'Number of students', 'Positie':'Position placed', 'Basisschool advies':'Education type'})
fig.update_yaxes(type='category')
fig.show()

### Number of choices student make

In [206]:
choice_2021[['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie']].isnull().sum()

Basisschool advies    0
Lotnummer             0
Geplaatst op          0
Positie               0
dtype: int64

In [207]:
choice_2021['num_choices'] = 21 - choice_2021.isnull().sum(axis=1)

In [208]:
choice_2021['num_choices'].describe().round()

count    7976.0
mean       10.0
std         4.0
min         1.0
25%         6.0
50%        12.0
75%        12.0
max        21.0
Name: num_choices, dtype: float64

In [209]:
fig = px.histogram(choice_2021, x="num_choices", histnorm='probability density', 
                    title='Distribution of number of schools each student fills in', 
                    labels={'num_choices':'Number of schools each student fills in'})
fig.show()

In [16]:
choice_2021['Voorkeur 1'].value_counts().nlargest(5)

Fons Vitae Lyceum - v.a. havo    269
Barlaeus Gymnasium - vwo         235
Fons Vitae Lyceum - vwo          208
Het Amsterdams Lyceum - vwo      205
Lumion - v.a. vmbo-t             198
Name: Voorkeur 1, dtype: int64

In [211]:
choice_2021['Voorkeur 2'].value_counts().nlargest(5)

Fons Vitae Lyceum - vwo           245
Fons Vitae Lyceum - v.a. havo     237
St. Nicolaaslyceum - v.a. havo    217
Het Amsterdams Lyceum - vwo       189
Barlaeus Gymnasium - vwo          184
Name: Voorkeur 2, dtype: int64

In [212]:
choice_2021['Voorkeur 3'].value_counts().nlargest(5)

Fons Vitae Lyceum - vwo           244
Fons Vitae Lyceum - v.a. havo     208
St. Nicolaaslyceum - v.a. havo    203
Het Amsterdams Lyceum - vwo       202
Spinoza20first - v.a. vmbo-t      200
Name: Voorkeur 3, dtype: int64

In [213]:
# choice_2021.groupby('Basisschool advies')['Voorkeur 1'].value_counts().to_dict()
# based on first choice:
most_popular_schools_per_level = {
('havo', 'Fons Vitae Lyceum - v.a. havo'): 114,
('havo/vwo', 'Fons Vitae Lyceum - v.a. havo'): 155,
('vmbo-b', 'Clusius College - v.a. vmbo-b'): 40,
('vmbo-b/k', 'Open Schoolgemeenschap Bijlmer - v.a. vmbo-b'): 41,
('vmbo-t', 'Lumion - v.a. vmbo-t'): 101,
('vmbo-t/havo', 'Lumion - v.a. vmbo-t'): 97,
('vwo', 'Barlaeus Gymnasium - vwo'): 235,
('vwo', 'Fons Vitae Lyceum - vwo'): 208,
('vwo', 'Het Amsterdams Lyceum - vwo'): 205,
('vwo', 'Hyperion Lyceum - vwo'): 153
}

In [19]:
235 / len(choice_2021[choice_2021['Basisschool advies'] == 'vwo']) * 100

10.262008733624455

In [223]:
choice_2020[['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie']].isnull().sum()

Basisschool advies    0
Lotnummer             0
Geplaatst op          0
Positie               0
dtype: int64

In [224]:
choice_2020.columns

Index(['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie',
       'Voorrang/Hardheid eerste voorkeur', 'Voorkeur 1', 'Voorkeur 2',
       'Voorkeur 3', 'Voorkeur 4', 'Voorkeur 5', 'Voorkeur 6', 'Voorkeur 7',
       'Voorkeur 8', 'Voorkeur 9', 'Voorkeur 10', 'Voorkeur 11', 'Voorkeur 12',
       'Voorkeur 13', 'Voorkeur 14', 'Voorkeur 15', 'Voorkeur 16',
       'Voorkeur 17', 'Voorkeur 18'],
      dtype='object')

In [7]:
choice_2020.head()

,Basisschool advies,Lotnummer,Geplaatst op,Positie,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18
9,vwo,7484,Cartesius Lyceum - vwo,12,Het 4e Gymnasium - vwo,Het Amsterdams Lyceum - vwo,Barlaeus Gymnasium - vwo,Montessori Lyceum Amsterdam - gymnasium - vwo,Ignatiusgymnasium - vwo,Vossius Gymnasium - vwo,Spinoza Lyceum - Gymnasium - vwo,Fons Vitae Lyceum - vwo,Cygnus gymnasium - vwo,St. Nicolaaslyceum - vwo,Hyperion Lyceum - vwo,Cartesius Lyceum - vwo,NaN,NaN,NaN,NaN,NaN,NaN
10,vwo,7574,Christelijke Scholengemeenschap Buitenveldert - v.a. vmbo-t,12,Het 4e Gymnasium - vwo,Ignatiusgymnasium - vwo,Het Amsterdams Lyceum - vwo,Barlaeus Gymnasium - vwo,Vossius Gymnasium - vwo,St. Nicolaaslyceum - Tweetalig Onderwijs - vwo,Fons Vitae Lyceum - vwo,Hervormd Lyceum Zuid - vwo,Gerrit van der Veen College - vwo,Cygnus gymnasium - vwo,Berlage Lyceum - Tweetalig onderwijs - vwo,Christelijke Scholengemeenschap Buitenveldert - v.a. vmbo-t,NaN,NaN,NaN,NaN,NaN,NaN
11,havo/vwo,7655,Christelijke Scholengemeenschap Buitenveldert - v.a. vmbo-t,12,Hervormd Lyceum Zuid - v.a. havo,Spinoza Lyceum - v.a. havo,St. Nicolaaslyceum - v.a. havo,Gerrit van der Veen College - v.a. havo,Fons Vitae Lyceum - v.a. havo,Berlage Lyceum - Tweetalig onderwijs - v.a. havo,Montessori Lyceum Amsterdam - v.a. havo,Geert Groote College - v.a. havo,Pieter Nieuwland College - v.a. havo,Metis Montessori Lyceum - v.a. havo,Spinoza20first - v.a. vmbo-t,Christelijke Scholengemeenschap Buitenveldert - v.a. vmbo-t,NaN,NaN,NaN,NaN,NaN,NaN
12,havo/vwo,7529,Damstede - v.a. havo,12,Geert Groote College - v.a. havo,Fons Vitae Lyceum - v.a. havo,Montessori Lyceum Amsterdam - v.a. havo,Gerrit van der Veen College - v.a. havo,Berlage Lyceum - Tweetalig onderwijs - v.a. havo,Pieter Nieuwland College - v.a. havo,Hervormd Lyceum Zuid - v.a. havo,Spinoza Lyceum - v.a. havo,Spinoza Lyceum - Muziekplus - v.a. havo,St. Nicolaaslyceum - Tweetalig Onderwijs - v.a. havo,St. Nicolaaslyceum - v.a. havo,Cartesius 2 - v.a. havo,Cartesius Lyceum - v.a. havo,Damstede - v.a. havo,NaN,NaN,NaN,NaN
13,havo,7638,Damstede - v.a. havo,12,Montessori Lyceum Amsterdam - v.a. havo,Gerrit van der Veen College - v.a. havo,Spinoza Lyceum - Muziekplus - v.a. havo,Fons Vitae Lyceum - v.a. havo,Geert Groote College - v.a. havo,Spinoza20first - v.a. vmbo-t,St. Nicolaaslyceum - v.a. havo,Cartesius Lyceum - v.a. havo,Vinse School - v.a. vmbo-t,Hervormd Lyceum Zuid - v.a. havo,Pieter Nieuwland College - v.a. havo,Damstede - v.a. havo,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
choice_2019.drop(['Unnamed: 24', 'Unnamed: 25'], axis=1, inplace=True)

In [11]:
choice_2019.head()

,Basisschool advies,Lotnummer,Geplaatst op,Positie,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19
12,"vmbo-b, met lwoo",104,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,"Vox-klassen - v.a. vmbo-b, met lwoo",Wellantcollege Linnaeus - v.a. vmbo-b,Montessori College Oost - v.a. vmbo-b,Huygens College - v.a. vmbo-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,"vmbo-b, met lwoo",120,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,Bredero Beroepscollege - v.a. vmbo-b,Montessori College Oost - v.a. vmbo-b,College De Meer - v.a. vmbo-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,"vmbo-k, met lwoo",122,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,Clusius College - v.a. vmbo-b,Bredero Beroepscollege - v.a. vmbo-b,"Vox-klassen - v.a. vmbo-b, met lwoo",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,vmbo-b/k,170,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,Clusius College - v.a. vmbo-b,Vox-klassen - v.a. vmbo-b,Bredero Beroepscollege - v.a. vmbo-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,"vmbo-b, met lwoo",280,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,Bredero Beroepscollege - v.a. vmbo-b,Clusius College - v.a. vmbo-b,Montessori College Oost - v.a. vmbo-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
choice_2020['num_choices'] = 18 - choice_2020.isnull().sum(axis=1)
choice_2020['num_choices'].describe().round()


count    7663.0
mean        9.0
std         4.0
min         1.0
25%         5.0
50%        12.0
75%        12.0
max        18.0
Name: num_choices, dtype: float64

In [13]:
choice_2020['num_choices'].mode()

0    12
Name: num_choices, dtype: int64

In [12]:
choice_2019['num_choices'] = 19 - choice_2019.isnull().sum(axis=1)
choice_2019['num_choices'].describe().round()

count    7568.0
mean        9.0
std         4.0
min         1.0
25%         5.0
50%        12.0
75%        12.0
max        19.0
Name: num_choices, dtype: float64

In [14]:
choice_2019['num_choices'].mode()

0    12
Name: num_choices, dtype: int64

### School popularity vs capacity

In [214]:
len(choice_2021[choice_2021['Voorkeur 1'] == 'Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b'])

39

In [215]:
# Num_choice_1: number of students who choose that school as their first choice 

school_2021['num_choice_1'] = [len(choice_2021[choice_2021['Voorkeur 1'] == i]) for i in school_2021['group']]
school_2021['num_choice_2'] = [len(choice_2021[choice_2021['Voorkeur 2'] == i]) for i in school_2021['group']]
school_2021['num_choice_3'] = [len(choice_2021[choice_2021['Voorkeur 3'] == i]) for i in school_2021['group']]


In [216]:
# acceptance rate = num_choice_1 / max_capacity, how competitive each school is
school_2021['acceptance_rate'] = (school_2021['max_capacity'] / school_2021['num_choice_1']).round(2)

In [217]:
school_2021.sort_values(by='acceptance_rate')

,school,group,max_capacity,num_choice_1,num_choice_2,num_choice_3,acceptance_rate
138,Metis Montessori Lyceum,Metis Montessori Lyceum - Technasium havo/vwo - v.a. havo,28,125,101,70,0.22
79,Metis Montessori Lyceum,Metis Montessori Lyceum - Technasium vwo - vwo,28,105,102,79,0.27
36,DENISE,DENISE - Denise TL - v.a. vmbo-t,7,23,25,27,0.30
37,Fons Vitae Lyceum,Fons Vitae Lyceum - v.a. havo,84,269,237,208,0.31
38,Fons Vitae Lyceum,Fons Vitae Lyceum - vwo,84,208,245,244,0.40
98,Spinoza Lyceum,Spinoza Lyceum - v.a. vmbo-t,56,138,128,151,0.41
137,Metis Montessori Lyceum,Metis Montessori Lyceum - Coderclass of Metisprofiel havo/vwo - v.a. havo,56,135,141,97,0.41
105,St. Nicolaaslyceum,St. Nicolaaslyceum - Tweetalig Onderwijs - v.a. havo,28,64,92,90,0.44
135,Alasca,Alasca - v.a. havo,56,114,94,81,0.49
136,Alasca,Alasca - vwo,56,109,76,33,0.51


In [219]:
choice_2019.columns

Index(['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie',
       'Voorkeurslijst', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')